In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from enum import Enum

class USBType(Enum):
    USB_A = "USB-A"
    USB_C = "USB-C"
    MICRO_USB = "Micro-USB"
    MINI_USB = "Mini-USB"
    UNKNOWN = "Unknown"

class USBClassifier:
    def __init__(self):

        self.debug_mode = True  # Set to True to display intermediate steps

    def preprocess_image(self, image_path):

        # Load image
        original = cv.imread(image_path)
        if original is None:
            raise Exception(f"Could not load image: {image_path}")

        result_img = original.copy()

        gray = cv.cvtColor(original, cv.COLOR_BGR2GRAY)

        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)

        blurred = cv.GaussianBlur(enhanced, (5, 5), 0)

        median_blurred = cv.medianBlur(blurred, 5)

        _, binary = cv.threshold(median_blurred, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

        white_pixel_ratio = np.sum(binary == 255) / binary.size
        if white_pixel_ratio > 0.5:
            binary = cv.bitwise_not(binary)

        if self.debug_mode:
            self.display_steps("Preprocessing", [
                ("Original", original),
                ("Grayscale", gray),
                ("Enhanced", enhanced),
                ("Blurred", blurred),
                ("Binary", binary)
            ])

        return original, gray, binary

    def extract_features(self, gray_img, binary_img):

        edges = cv.Canny(gray_img, 100, 200)

        # Morphological operations to clean up the binary image
        kernel = np.ones((5, 5), np.uint8)
        morphed = cv.morphologyEx(binary_img, cv.MORPH_CLOSE, kernel)
        morphed = cv.morphologyEx(morphed, cv.MORPH_OPEN, kernel)

        contours, _ = cv.findContours(morphed, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        if not contours:
            raise Exception("No contours found. Check the preprocessing steps.")

        usb_contour = max(contours, key=cv.contourArea)

        mask = np.zeros_like(gray_img)
        cv.drawContours(mask, [usb_contour], -1, 255, thickness=cv.FILLED)

        usb_region = cv.bitwise_and(gray_img, gray_img, mask=mask)

        rect = cv.minAreaRect(usb_contour)
        box = cv.boxPoints(rect)
        box = np.int32(box)

        width, height = rect[1]
        aspect_ratio = max(width, height) / min(width, height)

        area = cv.contourArea(usb_contour)
        perimeter = cv.arcLength(usb_contour, True)
        compactness = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0

        corner_img = gray_img.copy()
        corner_img = np.float32(corner_img)
        harris_corners = cv.cornerHarris(corner_img, blockSize=2, ksize=3, k=0.04)
        harris_corners = cv.dilate(harris_corners, None)

        corner_threshold = 0.01 * harris_corners.max()
        corner_count = np.sum(harris_corners > corner_threshold)

        hull = cv.convexHull(usb_contour, returnPoints=False)
        defects = None
        pin_count = 0

        try:
            if len(hull) > 3:  # Need at least 4 points for convexity defects
                defects = cv.convexityDefects(usb_contour, hull)
                if defects is not None:
                    # Filter significant defects (potential pins)
                    significant_defects = [defect for defect in defects if defect[0][3] > 1000]
                    pin_count = len(significant_defects)
        except:
            pass

        #For type C
        moments = cv.moments(usb_contour)
        hu_moments = cv.HuMoments(moments).flatten()
        symmetry_score = hu_moments[1]

        # Create a feature dictionary
        features = {
            'contour': usb_contour,
            'box': box,
            'aspect_ratio': aspect_ratio,
            'area': area,
            'perimeter': perimeter,
            'compactness': compactness,
            'corner_count': corner_count,
            'pin_count': pin_count,
            'symmetry_score': symmetry_score,
            'mask': mask,
            'usb_region': usb_region
        }

        if self.debug_mode:
            # Visualize features
            feature_vis = cv.cvtColor(gray_img, cv.COLOR_GRAY2BGR)
            cv.drawContours(feature_vis, [usb_contour], -1, (0, 255, 0), 2)
            cv.drawContours(feature_vis, [box], -1, (0, 0, 255), 2)

            # Visualize corners
            corner_vis = cv.cvtColor(gray_img, cv.COLOR_GRAY2BGR)
            corner_vis[harris_corners > corner_threshold] = [0, 0, 255]

            self.display_steps("Feature Extraction", [
                ("Edges", edges),
                ("Morphological Operations", morphed),
                ("Contour & Bounding Box", feature_vis),
                ("Harris Corners", corner_vis),
                ("USB Mask", mask),
                ("Extracted USB Region", usb_region)
            ])

        return features

    def classify_usb_type(self, features):

        aspect_ratio = features['aspect_ratio']
        compactness = features['compactness']
        corner_count = features['corner_count']
        pin_count = features['pin_count']
        symmetry_score = features['symmetry_score']

        # Classification logic based on geometric properties

        # USB-C typically has an aspect ratio close to 3:1 and is highly symmetric
        if 2.8 <= aspect_ratio <= 3.6 and symmetry_score < 0.0001:
            usb_type = USBType.USB_C
            confidence = 0.9
            reason = f"Classified as {usb_type.value}: aspect ratio = {aspect_ratio:.2f} (close to 3:1), " \
                     f"high symmetry (score = {symmetry_score:.8f})"

        # Micro-USB typically has an aspect ratio between 3.8:1 and 5:1 with a trapezoid shape
        elif 3.8 <= aspect_ratio <= 5.2 and compactness < 0.7:
            usb_type = USBType.MICRO_USB
            confidence = 0.85
            reason = f"Classified as {usb_type.value}: aspect ratio = {aspect_ratio:.2f} (close to 4:1), " \
                     f"compactness = {compactness:.2f} indicating trapezoid shape"

        # Mini-USB has a distinct shape with an aspect ratio around 2.5:1
        elif 2.2 <= aspect_ratio <= 2.8 and compactness < 0.75:
            usb_type = USBType.MINI_USB
            confidence = 0.8
            reason = f"Classified as {usb_type.value}: aspect ratio = {aspect_ratio:.2f} (close to 2.5:1), " \
                     f"compactness = {compactness:.2f}"

        # USB-A is more rectangular with an aspect ratio around 2:1
        elif 1.8 <= aspect_ratio <= 2.3 and corner_count > 100:
            usb_type = USBType.USB_A
            confidence = 0.85
            reason = f"Classified as {usb_type.value}: aspect ratio = {aspect_ratio:.2f} (close to 2:1), " \
                     f"significant corner features ({corner_count} corners)"

        # If no clear match, classify as unknown
        else:
            usb_type = USBType.UNKNOWN
            confidence = 0.0
            reason = f"Could not classify USB type. Features: aspect ratio = {aspect_ratio:.2f}, " \
                     f"compactness = {compactness:.2f}, corner count = {corner_count}, " \
                     f"symmetry score = {symmetry_score:.8f}"

        return usb_type, confidence, reason

    def visualize_result(self, original_img, features, usb_type, confidence, reason):
        result_img = original_img.copy()

        # Draw the contour and bounding box
        cv.drawContours(result_img, [features['contour']], -1, (0, 255, 0), 2)
        cv.drawContours(result_img, [features['box']], -1, (0, 0, 255), 2)

        # Add text with classification result
        font = cv.FONT_HERSHEY_SIMPLEX
        text_position = (10, 30)
        cv.putText(result_img, f"Type: {usb_type.value}", text_position, font, 1, (0, 255, 0), 2)

        text_position = (10, 70)
        cv.putText(result_img, f"Confidence: {confidence:.2f}", text_position, font, 1, (0, 255, 0), 2)

        # Add explanation for classification
        lines = reason.split(', ')
        y_pos = 110
        for line in lines:
            cv.putText(result_img, line, (10, y_pos), font, 0.6, (0, 200, 255), 2)
            y_pos += 30

        # Display key measurements
        y_pos += 20
        cv.putText(result_img, f"Aspect Ratio: {features['aspect_ratio']:.2f}", (10, y_pos), font, 0.6, (255, 0, 0), 2)
        y_pos += 30
        cv.putText(result_img, f"Compactness: {features['compactness']:.2f}", (10, y_pos), font, 0.6, (255, 0, 0), 2)

        plt.figure(figsize=(12, 10))
        plt.imshow(cv.cvtColor(result_img, cv.COLOR_BGR2RGB))
        plt.title(f"USB Classification: {usb_type.value}")
        plt.axis('off')
        plt.tight_layout()
        plt.show()

        return result_img

    def display_steps(self, stage_name, images):

        fig, axes = plt.subplots(1, len(images), figsize=(15, 5))
        fig.suptitle(stage_name)

        for i, (title, img) in enumerate(images):
            if len(images) == 1:
                ax = axes
            else:
                ax = axes[i]

            if len(img.shape) == 2:  # Grayscale
                ax.imshow(img, cmap='gray')
            else:  # Color
                ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

            ax.set_title(title)
            ax.axis('off')

        plt.tight_layout()
        plt.show()

    def process_image(self, image_path):

        original, gray, binary = self.preprocess_image(image_path)

        features = self.extract_features(gray, binary)

        usb_type, confidence, reason = self.classify_usb_type(features)

        result_img = self.visualize_result(original, features, usb_type, confidence, reason)

        return usb_type, confidence, reason, result_img

# Example usage
if __name__ == "__main__":
    classifier = USBClassifier()
    image_path = '/content/kkk.PNG'

    try:
        usb_type, confidence, reason, result_img = classifier.process_image(image_path)
        print(f"USB Type: {usb_type.value}")
        print(f"Confidence: {confidence:.2f}")
        print(f"Reason: {reason}")


        cv.imwrite('result.jpg', result_img)

    except Exception as e:
        print(f"Error processing image: {str(e)}")